
<img align="left" src = https://www.linea.org.br/wp-content/themes/LIneA/imagens/logo-header.png width=120 style="padding: 20px"> <br> 
<img align="right" src = https://jupyter.org/assets/homepage/hublogo.svg width=200 style="padding: 20px"> <br> 
<br>
<br>
<br>
<br>

# Minicurso JupyterHub - Caso Científico 2: Aglomerados de Galáxias

Andressa Wille

## Contexto e Objetivos

Neste notebook vamos explorar galáxias massivas da sequência vermelha de aglomerados.

Aglomerados de galáxias são grandes estruturas compostas por centenas a milhares de galáxias. Um exemplo famoso é o Aglomerado de Fornax (ACO S 373). Veja algumas das galáxias que o compõe na figura abaixo.

<img src="Fornax-Galaxy-Cluster.jpg">

Uma ferramenta importante para o estudo de aglomerados é o diagrama cor-magnitude (ou color-magnitude diagram, CMD). Nesse diagrama, é comum aparecerem duas regiões mais densas: a red sequence (sequência vermelha) e a blue cloud (nuvem azul).

<img src="color-magnitude_diagram.jpg">

Neste exemplo, vamos plotar o CMD de aglomerados com redshift entre 0.5 e 0.6. Depois, vamos selecionar algumas galáxias mais massivas da sequência vermelha para visualizar no Target Viewer.

## Bibliotecas e Environment

 Para criar um ambiente e instalar as bibliotecas necessárias a execução desse jupyter notebook, é necessário que os seguintes comandos sejam executados no terminal (conforme explicado nos slides):

```console
conda create -p $HOME/.conda/envs/tutorial
conda env list
conda activate tutorial # (ou source activate tutorial)
conda install pip numpy astropy scipy pandas sqlalchemy psycopg2 matplotlib seaborn ipykernel
conda install -c conda-forge fitsio
pip install astropandas
pip install dblinea
python -m ipykernel install --user --name=tutorial
# Refresh page (F5 no teclado não funciona, tem que ser no ícone mesmo).	
```

Realizada a configuração do ambiente, podemos prosseguir com a importação dos pacotes/módulos que serão utilizados nesse minicurso.

## Importando Módulos

In [ ]:
import matplotlib.pyplot as plt #Visualização de gráficos
import pandas as pd #Manejamento de tabelas e dataframes
import numpy as np #Pacote muntifuncional para uso matemático
from scipy.optimize import curve_fit #Função para ajuste de curvas

#Astropy é um módulo com inúmeras ferramentas úteis na vida dos astronomos/astrofísicos
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table

import astropandas as apd #Pacote necessário para o cross-match

#Comandos do JupyterNotebook para agilizar reimportações
%reload_ext autoreload
%autoreload 2

## Acesso ao banco de dados

Para acessar os dados do DES, vamos usar o módulo dblinea.

In [ ]:
from dblinea import DBBase
db = DBBase()
schema = "des_dr2"  
tablename = "coadd_objects"

## Seleção de dados

### WaZP

As galáxias membros de aglomerados que vamos estudar foram identificados por um algoritmo buscador de aglomerados chamado WaZP - Wavelet Z Photometric cluster finder -, aplicado sobre os dados do Dark Energy Survey. 
Nós utilizaremos a tabela do WaZP porque precisamos selecionar objetos no intervalo de redshift 0.5 a 0.6.

Essa tabela está disponível em http://dev.linea.org.br/~aguena//public_cats/wazp_y1/y1a1_dnf_wazp_v5.0.11.5943+39_members.fits

Você deve baixá-la antes de seguir os próximos passos do notebook.

In [ ]:
! wget http://dev.linea.org.br/~aguena//public_cats/wazp_y1/y1a1_dnf_wazp_v5.0.11.5943+39_members.fits 

Não iremos usar todos os dados da tabela, por isso faremos máscaras para pegar apenas uma região específica e no intervalo de redshift escolhido.

In [ ]:
wazp_members = Table.read('y1a1_dnf_wazp_v5.0.11.5943+39_members.fits', format='fits')

mask_z = (wazp_members['ZP'] > 0.5) & (wazp_members['ZP'] < 0.6)
mask_ra = (wazp_members['RA'] > 40) & (wazp_members['RA'] < 50)
mask_dec = (wazp_members['DEC'] > -50) & (wazp_members['DEC'] < -40)
mask = mask_z & mask_ra & mask_dec

wazp = wazp_members[mask]
wazp

Como não precisamos de todas as informações da tabela, vamos selecionar apenas as colunas necessárias.

In [ ]:
wazp_2 = wazp[['RA', 'DEC', 'ZP', 'mag_g', 'mag_r']]
wazp_2

Converter para pandas dataframe:

In [ ]:
wazp_df = Table.to_pandas(wazp_2)
wazp_df

#### Visualização espacial - WaZP

In [ ]:
ra =  wazp_df['RA']
dec =  wazp_df['DEC']

plt.hexbin(ra, dec, None,  mincnt=1, cmap='viridis', gridsize=[200,100])
plt.xlabel('ra (°)')
plt.ylabel('dec (°)')
plt.colorbar(label='density of points')
plt.title('Spacial distribution for WaZP members')
plt.grid()
plt.tight_layout()

#### CMD - WaZP

In [ ]:
mag1 = 'g'
mag2 = 'r'

plt.figure()
color = np.array(wazp_df[f'mag_{mag1}']) - np.array(wazp_df[f'mag_{mag2}'])
mag = np.array(wazp_df[f'mag_{mag1}'])               
#mask = (mag>14)&(mag<32)&(color<3)&(color>-3)

plt.hexbin(mag, color, None, mincnt=1, cmap='viridis', gridsize=[200,100])
cbar = plt.colorbar(label='density of points')
plt.xlabel('mag '+mag1)
plt.ylabel(f'{mag1}-{mag2}')
plt.title('Color-magnitude diagram for WaZP members')
plt.grid(True)
plt.tight_layout()

Já nesse plot é possível ver duas regiões de maior densidade, a red sequence e a blue cloud. Porém, esses dados ainda não precisos o suficiente, pois a cada medição de magnitude existe um erro associado.

Vamos filtrar nossos dados, eliminando galáxias com erro em magnitude maiores do que 0.1.
A tabela do WaZP não tem coluna de erro em magnitude, mas a tabela do DES DR2 sim. Faremos um cross-match.

### DES DR2 

O DES DR2 tem muitos dados, divididos em várias colunas. A descrição de cada coluna pode ser encontrada em:
https://des.ncsa.illinois.edu/releases/dr2/dr2-products/dr2-schema

A função a seguir nos mostra o nome das colunas disponíveis na tabela escolhida.

In [ ]:
db.get_table_columns(tablename, schema=schema)

A partir do dblinea é possível realizar queries, utilizando linguagem SQL. Vamos usar a função db.fetchall_df para coletar os dados.

Se quiser saber mais sobre uma função, basta buscar pelo help dela:

In [ ]:
help(db.fetchall_df)

A query que executaremos será a seguinte:

```sql
SELECT ra, dec, extended_class_coadd, mag_auto_g, mag_auto_r, magerr_auto_g, magerr_auto_r
FROM des_dr2.coadd_objects 
WHERE q3c_poly_query(ra, dec, ARRAY[{xlim[0]}, {ylim[1]}, {xlim[0]}, {ylim[0]}, {xlim[1]}, {ylim[0]}, {xlim[1]}, {ylim[1]}]) 
AND extended_class_coadd >= 2 
limit {l}
````

Precisamos das coordenadas (da mesma região que selecionamos anteriormente!), das magnitudes e erros em magnitude nas bandas g e r. Para selecionar objetos que são galáxias, não estrelas, usaremos o filtro extended_class_coadd >= 2.

In [ ]:
xlim = [40, 50]
ylim = [-50, -40]
l = 7500000
query = f"SELECT ra, dec, extended_class_coadd, mag_auto_g, mag_auto_r, magerr_auto_g, magerr_auto_r FROM des_dr2.coadd_objects WHERE q3c_poly_query(ra, dec, ARRAY[{xlim[0]}, {ylim[1]}, {xlim[0]}, {ylim[0]}, {xlim[1]}, {ylim[0]}, {xlim[1]}, {ylim[1]}]) AND extended_class_coadd >= 2 limit {l}"

A célula seguinte pode demorar alguns minutos para rodar.

In [ ]:
%%time
des_df = db.fetchall_df(query)
des_df

#### Visualização espacial - DES

In [ ]:
ra =  des_df['ra']
dec =  des_df['dec']

plt.hexbin(ra, dec, None,  mincnt=1, cmap='viridis', gridsize=[200,100])
plt.xlabel('ra (°)')
plt.ylabel('dec (°)')
plt.colorbar(label='density of points')
plt.title('Spacial distribution for DES data')
plt.grid(True)
plt.tight_layout()

#### Cross-match

O cross-match será feito pela função match do astropandas, que compara ascensão reta e declinação de dois dataframes diferentes.

In [ ]:
merged, info = apd.match(
    left=wazp_df, right=des_df,
    left_ra="RA", left_dec="DEC",
    right_ra="ra", right_dec="dec",
    threshold=1/3600)

In [ ]:
merged

#### CMD

Agora que temos a coluna de erro em magnitude, podemos fazer a máscara para eleminar objetos com erro muito grande, e plotar nosso diagrama cor-magnitude final.

In [ ]:
mag1 = 'g'
mag2 = 'r'

plt.figure()
color = np.array(merged[f'mag_auto_{mag1}']) - np.array(merged[f'mag_auto_{mag2}'])
mag = np.array(merged[f'mag_auto_{mag1}'])    

mask_err = merged[f'magerr_auto_{mag1}'] < 0.1

plt.hexbin(mag[mask_err], color[mask_err], None, mincnt=1, cmap='viridis', gridsize=[200,100])
cbar = plt.colorbar(label='density of points')
plt.xlabel('mag '+mag1)
plt.ylabel(f'{mag1}-{mag2}')
plt.title('Color-magnitude diagram')
plt.grid(True)
plt.tight_layout()

## Seleção das galáxias mais vermelhas

Vamos selecionar as galáxias mais vermelhas (cor maior) e mais brilhantes (magnitude menor).

In [ ]:
mask_red = (merged['mag_auto_g']<21) & (merged['mag_auto_g']-merged['mag_auto_r']>1.6)

In [ ]:
mag1 = 'g'
mag2 = 'r'

plt.figure()
color = np.array(merged[f'mag_auto_{mag1}']) - np.array(merged[f'mag_auto_{mag2}'])
mag = np.array(merged[f'mag_auto_{mag1}'])    

mask_err = merged[f'magerr_auto_{mag1}'] < 0.1 
mask = mask_err & mask_red

plt.plot(mag[mask], color[mask], 'ro')
plt.xlabel("mag "+mag1)
plt.ylabel(f"{mag1}-{mag2}")
plt.grid(True)
plt.tight_layout()

Quais são as coordenadas desses objetos?

In [ ]:
for ra, dec in zip(merged[mask]['RA'], merged[mask]['DEC']): 
    print(ra, ",", dec)

Agora que temos as coordenadas, podemos observar esses objetos no Science Server:

https://scienceserver.linea.org.br/target/#home

https://scienceserver.linea.org.br/sky/#sky